# 1. Rent 데이터셋 살펴보기

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
rent_df = pd.read_csv('/content/drive/MyDrive/머신러닝과 딥러닝/rent.csv')

FileNotFoundError: ignored

In [ ]:
rent_df

In [ ]:
rent_df.info()

* Posted On: 매물 등록 날짜
* BHK: 베드, 홀, 키친의 개수
* Rent: 렌트비
* Size: 집 크기
* Floor: 총 층수 중 몇층
* Area Type: 공용공간을 포함하는지, 집의 면적만 포함하는지
* Area Locality: 지역
* City: 도시
* Furnishing Status: 풀옵션 여부
* Tenant Preferred: 선호하는 가족 형태
* Bathroom: 화장실 개수
* Point of Contact: 연락할 곳

In [ ]:
rent_df.describe()

In [ ]:
7.810641e+04

In [ ]:
round(rent_df.describe(), 2)

In [ ]:
sns.displot(rent_df['BHK'])

In [ ]:
rent_df['BHK']

In [ ]:
sns.displot(rent_df['Rent'])

In [ ]:
rent_df['Rent'].sort_values()

In [ ]:
sns.boxplot(y=rent_df['Rent'])

In [ ]:
# rent_df.drop(1837, inplace=True)

In [ ]:
rent_df

In [ ]:
sns.boxplot(y=rent_df['Rent'])

In [ ]:
rent_df.isna().sum()

In [ ]:
rent_df.isna().mean()

In [ ]:
# Size에 있는 결측치 데이터를 삭제
rent_df.dropna(subset=["Size"])

In [ ]:
# 결측치가 있는 열을 삭제
rent_df.dropna(1) # BHK, Size 열이 모두 삭제, rent_df.drop('BHK', axis=1)

In [ ]:
rent_df = rent_df.reset_index(drop=True)

In [ ]:
# rent_df 데이터프레임에서 Size 컬럼이 NaN인 행을 모두 출력
rent_df[rent_df['Size'].isna()]

In [ ]:
na_index = rent_df[rent_df['Size'].isna()].index
na_index

In [ ]:
rent_df.iloc[na_index]

In [ ]:
# 결측치 처리
# 1. 결측 데이터가 전체 데이터에 비해 양이 굉장히 적을 경우 삭제하는 것도 방법
# 2. 결측치에 데이터를 채울 경우 먼저 boxplot을 확인하는 것이 좋음
sns.boxplot(y=rent_df['Size'])

In [ ]:
# boxplot을 확인 후 mean 보다는 median을 사용하는게 좋다고 판단!
rent_df.fillna(rent_df.median()).loc[na_index]

In [ ]:
na_index = rent_df[rent_df['BHK'].isna()].index
na_index

In [ ]:
rent_df['BHK'].fillna(rent_df['BHK'].median()).loc[na_index]

In [ ]:
rent_df = rent_df.fillna(rent_df.median())

In [ ]:
rent_df.isna().mean()

In [ ]:
rent_df.info()

In [ ]:
# Area Type은 텍스트 형태이기 때문에 모델에서 계산을 할 수 없음
# 라벨 인코딩을 통해 숫자로 변경
rent_df['Area Type'].value_counts()

In [ ]:
rent_df['Area Type'].unique()

In [ ]:
# 유니크한 종류의 개수
rent_df['Area Type'].nunique()

In [ ]:
for i in ['Floor', 'Area Type', 'Area Locality', 'City','Furnishing Status', 'Tenant Preferred', 'Point of Contact']:
    print(i, rent_df[i].nunique())

In [ ]:
rent_df.drop(['Posted On', 'Floor', 'Area Locality', 'Tenant Preferred', 'Point of Contact'], axis=1, inplace=True)

In [ ]:
rent_df.info()

In [ ]:
rent_df = pd.get_dummies(rent_df, columns=['Area Type', 'City', 'Furnishing Status'])

In [ ]:
rent_df.head()

In [ ]:
X = rent_df.drop('Rent', axis=1) # 독립변수
y = rent_df['Rent'] # 종속변수

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

In [ ]:
X_train.shape, y_train.shape

In [ ]:
X_test.shape, y_test.shape

# 2. 선형 회귀(Linear Regression)
* 데이터를 통해 가장 잘 설명할 수 있는 직선으로 데이터를 분석하는 방법
    * 단순 선형 회귀 분석(단일 독립변수를 이용)
    * 다중 선형 회귀 분석(다중 독립변수를 이용)

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lr = LinearRegression()

In [ ]:
lr.fit(X_train, y_train)

In [ ]:
pred = lr.predict(X_test)
# pred

# 3. 평가지표 만들기

### 3-1. MSE(Mean Squared Error)
* 예측값과 실제값의 차이에 대한 제곱에 대해 평균을 낸 값
* ${(\frac{1}{n})\sum_{i=1}^{n}(y_{i} - x_{i})^{2}}$
* 차이가 음수면 결과가 이상해지기 때문에 제곱을 함

In [ ]:
p = np.array([3, 4, 5])
act = np.array([1, 2, 3])

In [ ]:
def my_mse(pred, actual):
    return ((pred - actual) ** 2).mean()

In [ ]:
my_mse(p, act)

### 3-2. MAE(Mean Absolute Error)
* 예측값과 실제값의 차이에 대한 절대값에 대해 평균을 낸 값
* $(\frac{1}{n})\sum_{i=1}^{n}\left | y_{i} - x_{i} \right |$

In [ ]:
def my_mae(pred, actual):
    return np.abs(pred - actual).mean()

In [ ]:
my_mae(p, act)

### 3-3. RMSE(Root Mean Squared Error)
* 예측값과 실제값의 차이에 대한 제곱에 대해 평균을 낸 후 루트를 씌운 값
* $\sqrt{(\frac{1}{n})\sum_{i=1}^{n}(y_{i} - x_{i})^{2}}$

In [ ]:
def my_rmse(pred, actual):
    return np.sqrt(my_mse(pred, actual))

In [ ]:
my_rmse(p, act)

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [ ]:
mean_absolute_error(p, act)

In [ ]:
mean_squared_error(p, act)

In [ ]:
mean_squared_error(p, act, squared=False) # RMSE

### 3-4. 평가 지표 적용하기

In [ ]:
mean_squared_error(y_test, pred, squared=False)

In [ ]:
X_train.drop(1837, inplace=True)
y_train.drop(1837, inplace=True)

In [ ]:
lr.fit(X_train, y_train)

In [ ]:
pred = lr.predict(X_test)

In [ ]:
mean_squared_error(y_test, pred, squared=False)

In [ ]:
1717185779.0021067 - 41377.57030234839

# 4. log 활용하기

In [ ]:
a = [1,2,3,4,5]
b = [1, 10, 100, 1000, 10000]

In [ ]:
sns.lineplot(x=a, y=b)

In [ ]:
b_log = np.log(b)
b_log

In [ ]:
np.exp(b_log)

In [ ]:
y_train_log = np.log(y_train)

In [ ]:
lr.fit(X_train, y_train_log)

In [ ]:
pred = lr.predict(X_test)

In [ ]:
mean_squared_error(y_test, pred, squared=False)

* :1837 데이터를 삭제하기전 RMSE: 41438.9403701652
* 1837 데이터를 삭제한 후 RMSE: 41377.57030234839
* log를 활용하여 비선형으로 변경한 후 RMSE: 70418.55375574114

> 데이터의 분포가 비선형일 때 로그를 사용하는 것이 오차가 적으며, 선형일 때 로그를 사용하지 않는 것이 오차가 적음